In [18]:
from Crypto.Util.Padding import pad
import numpy as np
import os

import cpt

KEY_SIZE = 16
BLOCK_SIZE = 16

with open('flag', 'r') as f:
    flag = f.read().strip()
assert(len(flag) == 33)

test_msg = flag.encode()
test_msg = pad(test_msg, BLOCK_SIZE)
test_msg

b'dam{deg.aaa.sec,deg.minasd_qweec}\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f'

since the block size is 16 characters the message can be split up into 3 parts:

In [11]:
test_msg_ar = [test_msg[i:i+BLOCK_SIZE] for i in range(0, len(test_msg), BLOCK_SIZE)]
test_msg_ar = [np.array([ord(char) for char in block.decode('latin-1')]).reshape(4, 4).T for block in test_msg_ar]
test_msg_ar

[array([[100, 100,  97, 115],
        [ 97, 101,  97, 101],
        [109, 103,  97,  99],
        [123,  46,  46,  44]]),
 array([[100, 109, 115, 119],
        [101, 105, 100, 101],
        [103, 110,  95, 101],
        [ 46,  97, 113,  99]]),
 array([[125,  15,  15,  15],
        [ 15,  15,  15,  15],
        [ 15,  15,  15,  15],
        [ 15,  15,  15,  15]])]

The last array consists of only the ending curly bracket and the padding

---

The encrypt and decrypt functions they can be simplified by removing the conversion to strings and just working with the arrays

In [26]:
keys = np.array(bytearray(os.urandom(16))).reshape(4,4)
keys

def encrypt(state):
    for i in range(4):
        state = np.mod(state + keys[i], 256)
        state = np.roll(state, -1, axis=0)
        state = np.roll(state, -1, axis=1)
    return state
def decrypt(state):
    for i in range(3, -1, -1):
        state = np.roll(state, 1, axis=1)
        state = np.roll(state, 1, axis=0)
        state = np.mod(state - keys[i], 256)
    return state
def to_string(array):
    return ''.join([chr(char) for char in array.T.flatten()]).encode('latin-1')

assert((test_msg_ar[0] == decrypt(encrypt(test_msg_ar[0]))).all())

To make sure no mistakes were made compare it with the original function

In [27]:
to_string(encrypt(test_msg_ar[0]))

b"*'3A<=?\x06\x9a\x9a\x9ag\xfc\xee\xec\xb5"

In [28]:
cpt.encrypt(to_string(test_msg_ar[0]).decode('latin-1'), keys).encode('latin-1')


b"*'3A<=?\x06\x9a\x9a\x9ag\xfc\xee\xec\xb5"

The encryption function can be further simplified in the following way:

In [58]:
keys_simple = np.array([0, 0, 0, 0])
for i in range(4):
    for j in range(4):
        keys_simple[i] += keys[(j) % 4][((4-j+i) + 4) % 4]
keys_simple = np.mod(keys_simple, 256)
print(keys_simple)

def encrypt_simple(state):
    return np.mod(state + keys_simple, 256)

to_string(encrypt_simple(test_msg_ar[0]))

[198 216  57 137]


b"*'3A<=?\x06\x9a\x9a\x9ag\xfc\xee\xec\xb5"

In [59]:
original_iv = np.array(bytearray(os.urandom(16))).reshape(4,4)

In [60]:
iv = original_iv
ciphertext = [iv]
for i in range(0, 3):
    iv = encrypt_simple(np.bitwise_xor(iv, test_msg_ar[i]))
    ciphertext.append(iv)

for i in range(4):
    print(to_string(ciphertext[i]).hex())

08a6ed2d87271df9ecb0cddf846a7359
328d461cbb1a52afc60ae52a809899fe
1caee7f8ae4b14a6eea7f39480868526
2767aebd791cf3811ae135d4181213b2


Tring to recover the known key values from the last ciphertext and the last part of the message which should always be constant

In [64]:
from scipy.optimize import brute

expected = ciphertext[3]
state = np.bitwise_xor(ciphertext[2], test_msg_ar[2])

def opti_this(keys):
    return np.abs(np.mod(state + keys, 256) - expected).sum()

ranges = [(0, 256), (0, 256), (0, 256), (0, 256)]
result = brute(opti_this, ranges, full_output=True)
print(keys_simple)
result[0].round().astype(np.uint8)

[198 216  57 137]


array([198, 216,  57, 137], dtype=uint8)

Doing the same thing with the real values now

In [65]:
output = '791ec823a7a7499cbbd94f435f56cfd6bf2147fae01851f9343007a6f7a27c2634bfb02cd2c74d161015fa9e2970a6f6eb5261c527128c63c4bf9a36af083282'
output = [bytes.fromhex(output[i:i + BLOCK_SIZE * 2]) for i in range(0, len(output), BLOCK_SIZE * 2)]
output_ar = [np.array([ord(char) for char in block.decode('latin-1')]).reshape(4, 4).T for block in output]
output_ar

[array([[121, 167, 187,  95],
        [ 30, 167, 217,  86],
        [200,  73,  79, 207],
        [ 35, 156,  67, 214]]),
 array([[191, 224,  52, 247],
        [ 33,  24,  48, 162],
        [ 71,  81,   7, 124],
        [250, 249, 166,  38]]),
 array([[ 52, 210,  16,  41],
        [191, 199,  21, 112],
        [176,  77, 250, 166],
        [ 44,  22, 158, 246]]),
 array([[235,  39, 196, 175],
        [ 82,  18, 191,   8],
        [ 97, 140, 154,  50],
        [197,  99,  54, 130]])]

In [67]:
from scipy.optimize import brute

expected = output_ar[3]
state = np.bitwise_xor(output_ar[2], test_msg_ar[2])

def opti_this(keys):
    return np.abs(np.mod(state + keys, 256) - expected).sum()

ranges = [(0, 256), (0, 256), (0, 256), (0, 256)]

result = brute(opti_this, ranges, full_output=True)
keys_simple = result[0].round().astype(np.uint8)
keys_simple

array([162,  74, 165, 137], dtype=uint8)

In [81]:
def decrypt_simple(state):
    tmp = state - keys_simple
    return np.where(tmp < 0, tmp + 256, tmp)

msg = []
for i in range(2, -1, -1):
    msg.append(to_string(np.bitwise_xor(output_ar[i], decrypt_simple(output_ar[i+1]))))
''.join([m.decode('latin-1') for m in msg[::-1]])[:33]


'dam{1iN34R-B1O<K-<IpheR5_@R_WEaK}'